# 📘 Agentic 架构 12：图 / 世界模型记忆 (Graph / World-Model Memory)

欢迎来到对 AI 代理最强大记忆结构之一的详细探索：**基于图的世界模型**（Graph-based World Model）。这种架构超越了简单的文档检索或聊天历史，构建了对世界的结构化、相互关联的理解，非常类似于人类的语义记忆。

与将信息存储为孤立的文本块不同，基于图的代理将传入数据解析为**实体（节点）**和**关系（边）**。这创建了一个丰富、可查询的知识图谱。随后，代理可以通过遍历该图来回答复杂问题，发掘那些在非结构化文本中隐藏的洞见。

为了详细展示这一点，我们将构建一个**企业情报代理**。该代理将：
1.  **摄取非结构化报告：** 阅读关于公司、人员和产品的文本文档。
2.  **构建知识图谱：** 使用 LLM 提取实体（例如，`Company`、`Person`）和关系（例如，`ACQUIRED`、`WORKS_FOR`、`COMPETES_WITH`），并填充 Neo4j 图数据库。
3.  **回答复杂、多跳问题：** 利用图谱回答需要连接多条信息的问题，例如“*谁在收购了 BetaSolutions 的那家公司工作？*”——这类查询对于标准向量搜索来说极其困难。

### 定义
**图 / 世界模型记忆**（Graph / World-Model Memory）是一种代理式架构，将知识存储在结构化的图数据库中。信息以节点（实体，如人物、地点、概念）和边（它们之间的关系）形式表示。这创造了一个动态的“世界模型”，代理可以在其上进行推理。

### 高层工作流程

1.  **信息摄取：** 代理接收非结构化或半结构化数据（文本、文档、API 响应）。
2.  **知识提取：** 由 LLM 驱动的进程解析信息，识别关键实体以及连接它们的关系。
3.  **图更新：** 提取出的节点和边被添加到持久图数据库（如 Neo4j）中或在其中更新。
4.  **问答 / 推理：** 当被提问时，代理：
    a. 将自然语言问题转换为正式的图查询语言（例如，Neo4j 的 Cypher）。
    b. 对图执行查询，以检索相关的子图或事实。
    c. 将查询结果合成自然语言答案。

### 何时使用 / 应用场景
*   **企业知识助理：** 从内部文档中构建公司项目、员工和客户的可查询模型。
*   **高级研究助理：** 通过摄取研究论文，创建科学领域的动态知识库。
*   **复杂系统诊断：** 对系统的组件及其依赖关系建模，以诊断故障。

### 优势与劣势
*   **优势：**
    *   **结构化且可解释：** 知识高度组织化。可以展示导致答案的图中精确路径，从而解释答案。
    *   **支持复杂推理：** 擅长回答需要通过关系连接不同信息片的“多跳”问题。
*   **劣势：**
    *   **前期复杂度：** 需要定义良好的模式和健壮的提取过程。
    *   **保持图更新：** 管理更新、解决冲突信息以及随着时间推移修剪过时事实（知识生命周期管理）可能具有挑战性。

## 第零阶段：基础与设置

我们将安装库，包括 Neo4j 驱动程序，并配置运行环境。**关键的是，您必须拥有一个正在运行的 Neo4j 实例，并在 `.env` 文件中配置其凭证。**

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain_community neo4j

In [2]:
import os
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel as V1BaseModel

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Graph Memory (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "NEO4J_URI", "NEO4J_USERNAME", "NEO4J_PASSWORD"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## 第一阶段：构建图构建代理

这是摄取管道的核心部分。我们需要一个代理，能够读取非结构化文本，并以结构化格式提取实体和关系。我们将使用具有结构化输出能力（Pydantic）的 LLM 来充当我们的知识提取器。

In [3]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)

# Connect to our Neo4j database
try:
    graph = Neo4jGraph()
    # Clear the graph for a clean run
    graph.query("MATCH (n) DETACH DELETE n")
except Exception as e:
    console.print(f"[bold red]Failed to connect to Neo4j: {e}. Please check your credentials and connection.[/bold red]")
    graph = None

# Pydantic models for structured extraction (using LangChain's v1 BaseModel for compatibility with older structured output methods)
class Node(V1BaseModel):
    id: str = Field(description="Unique name or identifier for the entity.")
    type: str = Field(description="The type or label of the entity (e.g., Person, Company, Product).")

class Relationship(V1BaseModel):
    source: Node
    target: Node
    type: str = Field(description="The type of relationship (e.g., WORKS_FOR, ACQUIRED).")

class KnowledgeGraph(V1BaseModel):
    """A graph of nodes and relationships."""
    relationships: List[Relationship]

# The Graph Maker Agent
def get_graph_maker_chain():
    extractor_llm = llm.with_structured_output(KnowledgeGraph)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert at extracting information from text and building a knowledge graph. Extract all entities (nodes) and relationships from the provided text. The relationship type should be a verb in all caps, like 'WORKS_FOR' or 'ACQUIRED'."),
        ("human", "Extract a knowledge graph from the following text:\n\n{text}")
    ])
    return prompt | extractor_llm

graph_maker_agent = get_graph_maker_chain()
print("Successfully connected to Neo4j and defined the Graph Maker Agent.")

Successfully connected to Neo4j and defined the Graph Maker Agent.


## 第二阶段：摄取知识并构建世界模型

现在，我们将向代理提供一系列相关但独立的文档。代理将逐一处理这些文档，并逐步构建我们的企业知识图谱。这模拟了真实系统如何随着新信息的可用而随时间逐步学习的过程。

In [4]:
unstructured_documents = [
    "On May 15, 2023, global tech giant AlphaCorp announced its acquisition of startup BetaSolutions, a leader in cloud-native database technology.",
    "Dr. Evelyn Reed, a renowned AI researcher, has been the Chief Science Officer at AlphaCorp since 2021. She leads the team responsible for the QuantumLeap AI platform.",
    "Innovate Inc.'s flagship product, NeuraGen, is seen as a direct competitor to AlphaCorp's QuantumLeap AI. Meanwhile, Innovate Inc. recently hired Johnathan Miles as their new CTO."
]
for i, doc in enumerate(unstructured_documents):
    console.print(f"--- Ingesting Document {i+1} ---")
    try:
        kg_data = graph_maker_agent.invoke({"text": doc})
        if kg_data.relationships:
            graph.add_graph_documents(graph_documents=kg_data.relationships, include_source=False)
            console.print(f"[green]Successfully added {len(kg_data.relationships)} relationships to the graph.[/green]")
        else:
             console.print("[yellow]No relationships extracted.[/yellow]")
    except Exception as e:
        console.print(f"[red]Failed to process document: {e}[/red]")

console.print("--- ✅ Knowledge Graph Ingestion Complete ---")
console.print("\n--- Graph Schema ---")
console.print(graph.schema)

--- Ingesting Document 1 ---
Successfully added 1 relationships to the graph.
--- Ingesting Document 2 ---
Successfully added 2 relationships to the graph.
--- Ingesting Document 3 ---
Successfully added 2 relationships to the graph.
--- ✅ Knowledge Graph Ingestion Complete ---



--- Graph Schema ---


Node properties: [{'properties': [('id', 'STRING')], 'labels': ['Product']}, {'properties': [('id', 'STRING')], 'labels': ['Person']}, {'properties': [('id', 'STRING')], 'labels': ['Company']}]
Relationship properties: []
Relationships: [(:Company)-[:PRODUCES]->(:Product), (:Person)-[:WORKS_FOR]->(:Company), (:Product)-[:COMPETES_WITH]->(:Product), (:Company)-[:ACQUIRED]->(:Company)]


## 第三阶段：构建图查询代理

在我们的知识图谱已经填充完毕后，我们需要一个能够利用它的代理。这涉及一个**文本到 Cypher**的过程。代理将接收用户的自然语言问题，利用图模式作为上下文将其转换为 Cypher 查询，执行该查询，然后将结果合成易读的自然语言答案。

In [5]:
# LangChain has a built-in chain for this, but we'll inspect the components
# to understand how it works.
cypher_generation_prompt = ChatPromptTemplate.from_template(
    """You are an expert Neo4j Cypher query developer. Your task is to convert a user's natural language question into a valid Cypher query.
You must use the provided graph schema to construct the query. Do not use any node labels or relationship types that are not in the schema.
Return ONLY the Cypher query, with no additional text or explanations.

Graph Schema:
{schema}

User Question:
{question}
"""
)

cypher_response_prompt = ChatPromptTemplate.from_template(
    """You are an assistant that provides clear, natural language answers based on query results from a knowledge graph.
Use the context from the graph query result to answer the user's original question.

User Question: {question}
Query Result: {context}
"""
)

def query_graph(question: str) -> Dict[str, Any]:
    """The full Text-to-Cypher and synthesis pipeline."""
    console.print(f"\n[bold]Question:[/bold] {question}")
    
    # 1. Generate Cypher Query
    console.print("--- ➡️ Generating Cypher Query ---")
    cypher_chain = cypher_generation_prompt | llm
    generated_cypher = cypher_chain.invoke({"schema": graph.schema, "question": question}).content
    console.print(f"[cyan]Generated Cypher:\n{generated_cypher}[/cyan]")
    
    # 2. Execute Cypher Query
    console.print("--- ⚡ Executing Query ---")
    try:
        context = graph.query(generated_cypher)
        console.print(f"[yellow]Query Result:\n{context}[/yellow]")
    except Exception as e:
        console.print(f"[red]Cypher Query Failed: {e}[/red]")
        return {"answer": "I was unable to execute a query to find the answer to your question."}
    
    # 3. Synthesize Final Answer
    console.print("--- 🗣️ Synthesizing Final Answer ---")
    synthesis_chain = cypher_response_prompt | llm
    answer = synthesis_chain.invoke({"question": question, "context": context}).content
    
    return {"answer": answer}

print("Graph-Querying Agent defined successfully.")

Graph-Querying Agent defined successfully.


## 第四阶段：演示与分析

现在进入最终测试。我们将向代理提出从简单事实检索到复杂、多跳推理的问题，后者需要连接来自我们三个源文档的所有信息。

In [6]:
# Test 1: Simple fact retrieval (requires info from doc 2)
result1 = query_graph("Who works for AlphaCorp?")
console.print("\n--- Final Answer ---")
console.print(Markdown(result1['answer']))

# Test 2: Another simple fact retrieval (requires info from doc 1)
result2 = query_graph("What company did AlphaCorp acquire?")
console.print("\n--- Final Answer ---")
console.print(Markdown(result2['answer']))

# Test 3: The multi-hop reasoning question (requires info from all 3 docs)
result3 = query_graph("What companies compete with the products made by the company that acquired BetaSolutions?")
console.print("\n--- Final Answer ---")
console.print(Markdown(result3['answer']))


Question: Who works for AlphaCorp?
--- ➡️ Generating Cypher Query ---
Generated Cypher:
MATCH (p:Person)-[:WORKS_FOR]->(c:Company {id: 'AlphaCorp'}) RETURN p.id
--- ⚡ Executing Query ---
Query Result:
[{'p.id': 'Dr. Evelyn Reed'}]
--- 🗣️ Synthesizing Final Answer ---



--- Final Answer ---


Dr. Evelyn Reed works for AlphaCorp.


Question: What company did AlphaCorp acquire?
--- ➡️ Generating Cypher Query ---
Generated Cypher:
MATCH (:Company {id: 'AlphaCorp'})-[:ACQUIRED]->(acquired_company:Company)
RETURN acquired_company.id
--- ⚡ Executing Query ---
Query Result:
[{'acquired_company.id': 'BetaSolutions'}]
--- 🗣️ Synthesizing Final Answer ---



--- Final Answer ---


AlphaCorp acquired BetaSolutions.


Question: What companies compete with the products made by the company that acquired BetaSolutions?
--- ➡️ Generating Cypher Query ---
Generated Cypher:
MATCH (acquirer:Company)-[:ACQUIRED]->(:Company {id: 'BetaSolutions'})
MATCH (acquirer)-[:PRODUCES]->(product:Product)
MATCH (product)-[:COMPETES_WITH]->(competitor_product:Product)
MATCH (competitor_company:Company)-[:PRODUCES]->(competitor_product)
RETURN DISTINCT competitor_company.id
--- ⚡ Executing Query ---
Query Result:
[{'competitor_company.id': 'Innovate Inc.'}]
--- 🗣️ Synthesizing Final Answer ---



--- Final Answer ---


Innovate Inc. competes with the products made by the company that acquired BetaSolutions.

### 结果分析

演示突出了基于图的世界模型的深刻优势：

- 前两个问题是简单的查找。代理成功地将问题转换为 Cypher 查询，查询图谱，并找到了直接关系。

- 第三个问题是关键所在。一个标准的 RAG 代理在这里会失败。它可能找到关于收购的文档以及关于竞争对手的文档，但它难以将它们连接起来。它缺乏明确的关联结构，无法理解文档 1 中的“AlphaCorp”与文档 2 和 3 中的“AlphaCorp”是同一个实体。

- 然而，我们的基于图的代理却轻松解决了这个问题。我们可以直接从生成的 Cypher 查询中追踪其逻辑：
    1.  `MATCH (acquirer:Company)-[:ACQUIRED]->(:Company {id: 'BetaSolutions'})`：首先，找到收购 BetaSolutions 的公司（结果：AlphaCorp）。
    2.  `MATCH (acquirer)-[:PRODUCES]->(product:Product)`：接下来，找到该公司生产的产品（结果：QuantumLeap AI）。
    3.  `MATCH (product)-[:COMPETES_WITH]->(competitor_product:Product)`：然后，找到与该产品竞争的产品（结果：NeuraGen）。
    4.  `MATCH (competitor_company:Company)-[:PRODUCES]->(competitor_product)`：最后，找到生产竞争产品的公司（结果：Innovate Inc.）。

这种遍历关系并从不同来源合成信息的能力，正是图 / 世界模型架构的超能力。答案不仅仅是被检索出来，而是经过推理得出的。

## 结论

在本笔记本中，我们构建了一个完整的代理式系统，其核心围绕**图 / 世界模型记忆**。我们展示了完整的生命周期：摄取非结构化数据、使用 LLM 构建结构化知识图谱，然后利用该图谱回答需要真正推理的复杂、多跳问题。

这种架构相对于更简单的记忆系统而言，代表了能力上的显著飞跃。通过创建显式的、可查询的世界模型，我们赋予代理连接不同事实并发掘隐藏洞见的能力。虽然长期维护该图谱的挑战是真实的，但构建深度知识化且可解释的 AI 助理的潜力，使其成为现代代理式设计中最令人兴奋且最强大的模式之一。